In [ ]:
#%pip install git+https://github.com/huggingface/peft
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl 
#%pip install https://pypi.org/simple/ bitsandbytes

In [2]:
!rm -rf /kaggle/working/*

In [3]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer ,BitsAndBytesConfig
import torch

In [4]:
bnb_config = BitsAndBytesConfig(  
    load_in_4bit=True,
    bnb_4bit_quant_type= "nf4", # normal float : a new 4bit datatype adapted for weights that have been initialized using a normal distribution. 
    bnb_4bit_compute_dtype= torch.bfloat16, # Set the computation data type to bfloat16 instead of default float32
    bnb_4bit_use_double_quant= True, # Nested quantization : where the quantization constants from the first quantization are quantized again. - no performance lost
)

In [ ]:
%%time

config = PeftConfig.from_pretrained("FarahOU/adapt-llm-Timesheet-Fr-80xr512-1")
model_new = AutoModelForCausalLM.from_pretrained("AdaptLLM/finance-chat", device_map="cuda", quantization_config=bnb_config)
model_new = PeftModel.from_pretrained(model_new, "FarahOU/adapt-llm-Timesheet-Fr-80xr512-1") #.to("cuda")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("FarahOU/adapt-llm-Timesheet-Fr-80xr512-1", trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

# List input : Input Preparation 

In [6]:
def formatList(L):
    L_res=[]
    i=0

    while i < len(L) : 
        strr="{ " + L[i] + " },"
        j=i+1
        while (j< len(L)) and (L[i].split(",")[0]== L[j].split(",")[0]): 
            strr+= "{ " + L[j].split('"Explanation":')[0] + " },"
            j+=1
           
        L_res.append(strr + ' "Explanation":' + L[j-1].split('"Explanation":')[1])
        i=j
        
    return L_res


import pandas as pd
data=pd.read_csv("/kaggle/input/timehseett/Timesheet_test_30_anonym.csv")

json_data = data.to_json(orient='records')

L=json_data.split('},{')

L[0]=L[0][2:]
L[-1]=L[-1][:-2]

L_test_format = formatList(L)

for i in range(len(L_test_format)) : 
    Scenario = L_test_format[i].split('"Explanation":"')
    if (len(Scenario)<2) :
        Scenario.append("null")
    L_test_format[i] = Scenario[0] + '\n "Explanation":"' + Scenario[1]
    

# Batch function Definition & Generation

In [7]:
# def batch_generation(L) :
    
#     L_result=[]
#     for i in range(len(L)) :
#         user_input = (f'''{L[i]} 
        
#             Instruction : Generate a Timesheet Fraud scenario :           
#             ''' +'''
            
#             {
#                "PersonID":"",
#                "SOURCE":"",
#                "Payroll_Period":"",
#                "Absence_Reason":"",
#                "Absence_CODE":"",
#                "Absence_Start_Date":"",
#                "Absence_End_Date":"",
#                "Absence_Duration":"",
#                "Acquired_Leave_days":"",
#                "Taken_Leave_days":"",
#                "Remaining_Leave_days":"",
#                "Following_year_reported_Leave":"",
#                "Explanation":"",
#             }

#             Completion :  
            
#             ''')
#         initial_prompt = ''' Timesheet fraud involves the manipulation or falsification of time records by employees to claim more hours than they have actually worked, leading to inflated compensation. 
#         Note that the actual year we are currently at is : 2024. 
#         I want it to be in json format.
       
#         '''

#         prompt = f"<s>[INST] <<SYS>>{initial_prompt}<</SYS>>\n\n{user_input} [/INST]"


#         inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to(model_new.device) #to("cuda") #.to(model_new.device)
#         outputs =model_new.generate(input_ids=inputs, 
#                              max_new_tokens=3700, #2500 default ; #1200 for 3 examples 
#                              do_sample=True,
#                              temperature=1.3
#                             )[0]

#         answer_start = int(inputs.shape[-1])
#         pred = tokenizer.decode(outputs[answer_start:], skip_special_tokens=True)
#         L_result.append(pred)
        
#         # save str in file format
#         print(user_input)
#         file_name = 'Assis_5_Temp3_Timesheet' + str(i) + '.txt'
#         with open( file_name , 'w') as f:
#             f.write(pred)
#         print(f' \n file {i} saved')
        
#         print(pred)
#         print('\n ------------------')
    
#     return L_result


In [8]:
def batch_generation(L) :
    
    L_result=[]
    for i in range(len(L)) :
        user_input = (f'''{L[i]} 
        

            Instruction : Generate a Timesheet Fraud scenario :           
            ''' +'''

            {
                   PersonID:
                   SOURCE:
                   Payroll_Period:
                   Absence_Reason:
                   Absence_CODE:
                   Absence_Start_Date:
                   Absence_End_Date:
                   Absence_Duration:
                   Acquired_Leave_days:
                   Taken_Leave_days:,
                   Remaining_Leave_days:
                   Following_year_reported_Leave:
                 Explanation:
            }

            Completion :  


        Generate a clean scenario, without irrelevant symbols and text in between.
        Explain your reasoning for the scenario you're going to provide, step by step.
            ''')

        initial_prompt = ''' Timesheet fraud involves the manipulation or falsification of time records by employees to claim more hours than they have actually worked, leading to inflated compensation. 
        Note that the actual year we are currently in is : 2024. 
        I want it to be in json format.

        Show clean output, don't print out successive sequences of ' },  },  }, '.


        You can characterize one scenario as having the same PersonID. The scenario can have the format of 1n or 2 or 3 json objects, because it corresponds to historical absence and taken leave data of each employee, there is one explanation of the fraud scenario provided at the end of the sceanrio that takes into account the history of acquired leave days, remaining and taken leave days.
        Acquired leave days are Allowed vacation days that employees can cumulate as they work in the company. Their values get upgraded by 1.2days or 1.5 days depending on the employee post. Their values get updated each payroll period, meaning, each month.
        Taken leave days are Leave days that the employees have already consumed, for reasons like sick leave, day of rest, etc.
        Remaining leave days are the difference between acquired leave days and Taken leave days.

        You will be a Timesheet Fraud Scenrios Generator. You will Generate Logical and reasonable Timesheet Fraud Scenarios in json format, featuring the Keys in the example down below.

        '''
        prompt = f"<s>[INST] <<SYS>>{initial_prompt}<</SYS>>\n\n{user_input} [/INST]"


        inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to(model_new.device) #to("cuda") #.to(model_new.device)
        outputs =model_new.generate(input_ids=inputs, 
                             max_new_tokens=3700,  #1200 for 3 examples 
                             do_sample=True,
                             temperature=1.6
                            )[0]

        answer_start = int(inputs.shape[-1])
        pred = tokenizer.decode(outputs[answer_start:], skip_special_tokens=True)
        L_result.append(pred)
        
        # save str in file format
        print(user_input)
        file_name = 'Assis_55_Temp13_Timesheet' + str(i) + '.txt'
        with open( file_name , 'w') as f:
            f.write(pred)
        print(f' \n file {i} saved')
        
        print(pred)
        print('\n ------------------')

    return L_result


In [9]:
# # converting to final format
# def readable_csv(L_result,k=0) : 
#     str_json = ''
#     for string in L_result :
#         json_output = json.loads(string)
#         str_json+=str(json_output)
#         if str_json[-1] != ',' :
#             str_json+=','
        
#     df = pd.read_json(str_json[:-1])
#     df.to_csv(f"final_output{k}.csv")
    
#     return str_json
    

## Generation of (List size) 

In [ ]:
%%time
batch_generation(L_test_format)

# Mass Generation (50 X list size)

In [11]:
# L_total=[]
# for j in range(5) : 
#     res = batch_generation(L,j)
#     L_total.append(res)
    